# Insights na TechGrow

## 📦 Contexto
Você faz parte do time de dados da TechGrow, uma empresa SaaS que oferece soluções de gestão para pequenos negócios.

Nos últimos meses, a liderança percebeu um aumento preocupante na taxa de cancelamento (churn).

O time de produto quer descobrir por que isso está acontecendo e quais clientes estão em risco.

Você recebeu um conjunto de dados com informações de clientes, uso da plataforma e histórico de faturamento.

Sua missão é criar uma análise exploratória completa, treinar modelos preditivos e interpretar os resultados para gerar insights acionáveis.

## 🧩 Desafio
1. Importe e explore os dados
- Carregue o arquivo `Caso_Pratico_base_churn_saas.csv`.
- Observe o formato, colunas disponíveis e tipos de variáveis.
- Verifique valores nulos e estatísticas descritivas.
- Crie visualizações para entender a distribuição das variáveis e o comportamento dos clientes que cancelaram (`churn == 1`).
2. Trate e prepare os dados
- Aplique encoding em variáveis categóricas.
- Padronize ou normalize as colunas numéricas, se necessário.
- Verifique correlações relevantes e elimine colunas redundantes.
3. Separe os dados em treino e teste
- Utilize 70% para treino e 30% para teste.
- Defina `churn` como variável-alvo (`target`).
4. Treine os modelos
- Comece com um modelo baseline: Regressão Logística.
- Em seguida, treine e compare com Random Forest e XGBoost.
5. Avalie os resultados
- Use métricas de classificação:
  - `accuracy`
  - `precision`
  - `recall`
  - `f1-score`
  - `roc_auc_score`
- Plote a matriz de confusão e a curva ROC.
6. Interprete e apresente os resultados
- Quais variáveis mais impactam o churn?
- Seu modelo consegue identificar bem os clientes que vão cancelar?
- Quais ações você recomendaria ao time de produto?

## 🧠 O que você vai praticar
- Análise exploratória e preparação de dados
- Criação de modelos supervisionados de classificação
- Avaliação de performance de modelos
- Interpretação de resultados e storytelling com dados

## 💜 Dica da Rocketseat
O foco deste desafio não é apenas encontrar o modelo com melhor acurácia, mas entender o porquê.

Explique suas decisões, documente suas observações e interprete o impacto de cada variável.

Pense como uma pessoa de dados que ajuda o time a tomar decisões baseadas em evidências. ✨

## 📂 Dataset
Arquivo: `Caso_Pratico_base_churn_saas.csv`

Contém colunas relacionadas a:

- Perfil do cliente (tempo de contrato, localização, tipo de plano)
- Comportamento de uso (número de logins, volume de ações, tickets de suporte)
- Histórico financeiro (valor mensal, inadimplência, descontos aplicados)
- Variável alvo: `churn` (1 = cancelou, 0 = manteve o serviço)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("Caso_Pratico_base_churn_saas.csv")

df.head()
df.info()
df.describe()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
sns.countplot(x='churn', data=df)
plt.title("Distribuição do Churn")
plt.show()

df['churn'].value_counts(normalize=True)

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns

for col in num_cols:
    plt.figure(figsize=(6,3))
    sns.histplot(df[col], kde=True)
    plt.title(col)
    plt.show()

In [ ]:
for col in num_cols:
    plt.figure(figsize=(6,3))
    sns.boxplot(x='churn', y=col, data=df)
    plt.title(f"{col} vs churn")
    plt.show()

In [ ]:
cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    plt.figure(figsize=(6,3))
    sns.countplot(x=col, hue='churn', data=df)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
X = df.drop("churn", axis=1)
y = df["churn"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

y_pred_log = log_model.predict(X_test_scaled)
y_prob_log = log_model.predict_proba(X_test_scaled)[:,1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:,1]

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    eval_metric="logloss",
    random_state=42
)

xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, RocCurveDisplay

In [ ]:
print("Logistic Regression")
print(classification_report(y_test, y_pred_log))
print("ROC AUC:", roc_auc_score(y_test, y_prob_log))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_xgb),
            annot=True, fmt='d')
plt.title("Matriz de Confusão - XGBoost")
plt.show()

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_prob_xgb)
plt.show()

In [ ]:
importances = pd.Series(
    rf_model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

importances.head(10)

In [ ]:
coef = pd.Series(
    log_model.coef_[0],
    index=X.columns
).sort_values(ascending=False)